# 北尾早霧・砂川武貴・山田知明『定量的マクロ経済学と数値計算』日本評論社
## 第2章：2期間モデル・3期間モデルと数値計算の概観

## 2.3.2 グリッドの効率的な取り方
* このNotebookを読む前に`quantmacro_chapter2_1.ipynb`に目を通すことをお勧めします

### 1. 等分でグリッドを計算するための関数

In [14]:
def grid_uni(grid_min:float, grid_max:float, num_grid:int):
    """
    grid_uni(grid_min::Float64, grid_max::Float64, num_grid::Int64)

    Purpose:
    Generate uniform grid from "grid_min" to "grid_max".
    "linspace" in Matlab.

    Input:
    grid_min → minimum of grid
    grid_max → maximum of grid
    num_grid → # grid

    Output:
    grid → Vector of num_grid × 1
    """

    grid = np.zeros(num_grid)
    increment = (grid_max-grid_min)/(num_grid-1)

    for i in range(num_grid):
        grid[i] = i*increment + grid_min 
    
    # avoid rounding error
    if grid[num_grid-1] != grid_max:
        grid[num_grid-1] = grid_max
    
    return grid

### 2. 指数的にグリッドを計算するための関数(1)
* このコードは[Econ-ARK](https://econ-ark.org/)の元になったC.Carroll氏のMATLABコードを参考にしています。

In [15]:
def grid_dbl(grid_min:float, grid_max:float, num_grid:int):
    """
    function grid_dbl(grid_min::Float64, grid_max::Float64, num_grid::Int64)

    Purpose:
    Generate a double exponentially-spaced grid.

    Input:
    grid_min → minimum of grid
    grid_max → maximum of grid
    num_grid → # grid

    Output:
    grid → Vector of num_grid × 1
    """

    grid = np.zeros(num_grid)
    lmax = np.log(np.log(grid_max+1) + 1)
    lmin = np.log(np.log(grid_min+1) + 1)
    mesh = grid_uni(lmin,lmax,num_grid)

    for i in range(num_grid):
        grid[i] = np.exp(np.exp(mesh[i])-1) - 1
    
    return grid

### 3. 指数的にグリッドを計算するための関数(2)
* このコードは[Econ-ARK](https://econ-ark.org/)の元になったC.Carroll氏のMATLABコードを参考にしています。

In [16]:
def grid_tri(grid_min:float, grid_max:float, num_grid:int):
    """
    function grid_tri(grid_min::Float64, grid_max::Float64, num_grid::Int64)

    Purpose:
    Generate a triple exponentially-spaced grid.

    Input:
    grid_min → minimum of grid
    grid_max → maximum of grid
    num_grid → # grid

    Output:
    grid → Vector of num_grid × 1
    """

    grid = np.zeros(num_grid)
    lmax = np.log(np.log(np.log(grid_max+1) + 1) + 1)
    lmin = np.log(np.log(np.log(grid_min+1) + 1) + 1)
    mesh = grid_uni(lmin,lmax,num_grid)

    for i in range(num_grid):
        grid[i] = np.exp(np.exp(np.exp(mesh[i]) - 1) - 1) - 1
    
    return grid

### 4. Maliar et al. (2010)で紹介されているグリッド計算方法

In [27]:
def grid_mmv(grid_min:float, grid_max:float, θ:float, num_grid:int):
    """
    grid_mmv(grid_min::Float64, grid_max::Float64, θ::Float64, num_grid::Int64)

    Purpose:
    Generate grids based on the idea by
    Maliar, Maliar and Valli (2010):
    "Solving the Incomplete Markets Model with Aggregate Uncertainty using the Krusell-Smith Algorithm," Journal of Economic Dynamics and Control

    Input:
    grid_min → minimum of grid
    grid_max → maximum of grid
    num_grid → # grid

    Output:
    grid → Vector of num_grid × 1
    """

    # Equation (7) in Maliar et al. (2010,JEDC)
    tmp = np.zeros(num_grid)
    
    for i in range(num_grid):
        tmp[i] = (i/(num_grid-1))**θ * grid_max
    
    # adjust to [grid_min, grid_max]
    grid = np.zeros(num_grid)
    grid[1] = grid_min

    for i in range(1,num_grid):
        grid[i] = grid[i-1] + (tmp[i]-tmp[i-1]) / grid_max*(grid_max-grid_min)
    
    return grid

## 試しにグリッドを計算してみよう

In [28]:
import numpy  as np 

nw = 11 # グリッドの数
w_min = 0.0 # グリッドの下限
w_max = 1.0 # グリッドの上限

In [29]:
grid_uni(w_min, w_max, nw)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [30]:
grid_dbl(w_min, w_max, nw)

array([0.        , 0.05555854, 0.11746604, 0.18665559, 0.264229  ,
       0.35149298, 0.45000408, 0.56162512, 0.68859622, 0.83362476,
       1.        ])

In [31]:
grid_tri(w_min, w_max, nw)

array([0.        , 0.04514782, 0.09669846, 0.15592185, 0.22441776,
       0.30422284, 0.39795986, 0.50904859, 0.64200895, 0.80290401,
       1.        ])

In [32]:
θ = 2.0
grid_mmv(w_min, w_max, θ, nw)

array([0.  , 0.01, 0.04, 0.09, 0.16, 0.25, 0.36, 0.49, 0.64, 0.81, 1.  ])

In [34]:
θ = 4.0
grid_mmv(w_min, w_max, θ, nw)

array([0.000e+00, 1.000e-04, 1.600e-03, 8.100e-03, 2.560e-02, 6.250e-02,
       1.296e-01, 2.401e-01, 4.096e-01, 6.561e-01, 1.000e+00])